In [7]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

%matplotlib inline
import matplotlib.pyplot as plt


In [8]:
items = pd.read_csv('items.csv')
items.head()

,Name,item_category,item_brand,item_weight,item_type
0,зубная паста лакалют актив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
1,зубная паста лакалют сенситив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
2,зубная паста лесной бальзам ромашка и облепиха...,"Красота, гигиена, бытовая химия",лесной бальзам,NaN,зубная паста
3,зубная паста лесной бальзам с экстрактом коры ...,"Красота, гигиена, бытовая химия",лесной бальзам,NaN,зубная паста
4,"колбаса молочная вязанка вареная мини 0,5кг (с...","Птица, мясо, деликатесы",вязанка,5кг,колбаса


In [9]:
items['item_type'].unique()

array(['зубная паста', 'колбаса', 'кофе', 'напиток', 'пюре', 'сыр', 'чай',
       'шампунь', 'шоколад', 'корм'], dtype=object)

In [10]:
purchases = pd.read_csv('purchases.csv')
purchases.head()

,user_id,item
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо..."
1,ba82ad84-3a19-5a91-8e1e-7fd87628afb4,пюре тема говядина с гречкой с 8 месяцев
2,74a2856d-f0ec-59a6-89f3-1f80b294e852,колбаса микоян сервелат кремлевский варено-коп...
3,bbd344a2-a095-5910-b2e7-66d6971e1b76,сыр колбасный город сыра гурманский копченый г...
4,5ec2fc2a-c327-5f59-b920-fda229cd8175,"колбаса папа может мясная вареная, 500г"


In [11]:
purchases = pd.merge(purchases, items, left_on='item', right_on='Name')


In [12]:
purchases.drop('item', axis=1, inplace=True)
purchases.head()

,user_id,Name,item_category,item_brand,item_weight,item_type
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
1,b4a10859-3f8c-5dc1-8d5d-5977f9aa8bde,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
2,464053f2-ead4-500e-8486-9d5d66c1bbd7,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
3,baf7d53b-3170-5984-b05c-c5d2b8788d57,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
4,ef6beb0e-a09e-5372-aa91-eb467f117aa8,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад


In [13]:
purchases['y'] = purchases['item_type'].apply(lambda x: 1 if x == 'чай' else 0)

In [14]:
X = purchases.groupby(['user_id']).agg({
    'Name': lambda x: list(x),
    'y': lambda x: max(x)
})

X['user_id'] = [i for i in X.index.values]
X.columns = ['purchases', 'y', 'user_id']
X = X[['user_id', 'purchases', 'y']]
X.index = range(len(X))
X.head(3)

,user_id,purchases,y
0,00002f01-66e4-5ab8-8d1a-1562a4ddd418,[зубная паста splat stress off антистресс 75мл...,0
1,0000fed8-b063-51ef-8ca4-c42c5bd022ad,[шоколад schogetten black & white молочный с к...,0
2,0004cfe8-bcb2-5a2c-904b-643e0469cbe3,"[шоколад воздушный темный 85г, сыр белебеевски...",0


In [15]:
X['purchases'].values[0]

['зубная паста splat stress off антистресс 75мл',
 'зубная паста splat junior карамельная груша 55мл',
 'зубная паста president kids lollipop со вкусом леденца 50мл',
 'корм purina one для собак с курицей',
 'зубная паста splat special wonder white, 75мл']

In [16]:
import nltk
from nltk.corpus import stopwords
import re
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [17]:
# Русские стоп слова из nltk
nltk.download('stopwords')
stopword_ru = stopwords.words('russian')
stopword_eng = stopwords.words('english')
print(len(stopword_ru))

with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru = additional_stopwords + stopword_eng + stopword_ru
len(stopword_ru)

151


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


955

In [18]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [19]:
X['purchases'] = X['purchases'].apply(lambda x: clean_text(x))

<ipython-input-18-898b573b2830>:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


In [20]:
X['purchases'] = X['purchases'].apply(lambda x: lemmatization(x))

In [21]:
X['purchases'] = X['purchases'].apply(lambda x: ' '.join(x))

In [22]:
X.head(5)

,user_id,purchases,y
0,00002f01-66e4-5ab8-8d1a-1562a4ddd418,зубной паста splat stress антистресс мл зубной...,0
1,0000fed8-b063-51ef-8ca4-c42c5bd022ad,шоколад schogette black white молочный кусочек...,0
2,0004cfe8-bcb2-5a2c-904b-643e0469cbe3,шоколад воздушный тёмный сыр белебеевский башк...,0
3,000b8172-b96d-5c99-a418-fe1ca156bee1,шампунь pa te prov интенсивный восстановление ...,0
4,000bf80e-219c-53b7-a000-6c3474c2bd14,шоколад schogette black white молочный кусочек...,0


In [23]:
X

,user_id,purchases,y
0,00002f01-66e4-5ab8-8d1a-1562a4ddd418,зубной паста splat stress антистресс мл зубной...,0
1,0000fed8-b063-51ef-8ca4-c42c5bd022ad,шоколад schogette black white молочный кусочек...,0
2,0004cfe8-bcb2-5a2c-904b-643e0469cbe3,шоколад воздушный тёмный сыр белебеевский башк...,0
3,000b8172-b96d-5c99-a418-fe1ca156bee1,шампунь pa te prov интенсивный восстановление ...,0
4,000bf80e-219c-53b7-a000-6c3474c2bd14,шоколад schogette black white молочный кусочек...,0
...,...,...,...
31995,fff4348d-4028-5775-9616-9acdd6fbb0aa,зубной паста лесной бальзам дчувствительный зу...,0
31996,fff575de-27be-54e3-bb84-dfc65eeb0cb2,зубной паста colgate optic white искриться бел...,0
31997,fff5a1c6-3aa2-5684-b2d6-7037aad1736f,шоколад россия щедрый душа золотой марк дуэт г...,0
31998,fff83aec-4b5a-5bff-95bc-3b4543d97cf6,колбаса рублёвский краковский полукопчёный зуб...,0


In [24]:
X.drop('user_id', axis=1, inplace=True)

In [25]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print(f"f1: {f1 * 100.0:.2f}%") 
    rec = recall_score(y_test, y_predict, average='binary')
    print(f"recall: {rec * 100.0:.2f}%") 
    prc = precision_score(y_test, y_predict, average='binary')
    print(f"precision: {prc * 100.0:.2f}%" ) 

In [26]:
# соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline1 = Pipeline([('purchases_selector', FeatureSelector(column='purchases')), 
                     ('purchases_tfidf', TfidfVectorizer()), 
                     ('clf', RandomForestClassifier())])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, X['y'], test_size=0.33, random_state=1)
pipeline1.fit(X_train, y_train)
y_predict1 = pipeline1.predict(X_test)
evaluate_results(y_test, y_predict1)

Classification results:
f1: 96.30%
recall: 93.26%
precision: 99.54%


In [28]:
X['y'].value_counts()

0    30640
1     1360
Name: y, dtype: int64

In [29]:
p = 0.5
pos = int(np.ceil(p * len(X[X['y'] == 1])))
X_pos = X[X['y'] == 1][:pos]
X_unlabeled = X[X['y'] == 0][:pos]
X_test = X.sample(frac=1)[:pos*2]
y_test = X_test['y']
X_test.drop('y', axis=1, inplace=True)

In [30]:
X_pos.shape, X_unlabeled.shape, X_test.shape

((680, 2), (680, 2), (1360, 1))

In [31]:


pipeline2 = Pipeline([('purchases_selector', FeatureSelector(column='purchases')), 
                     ('purchases_tfidf', TfidfVectorizer()), 
                     ('clf', RandomForestClassifier())])

In [35]:
X = pd.concat([X_pos, X_unlabeled]).sample(frac=1)

In [36]:
pipeline2.fit(X.drop(columns=['y']), 
          X['y'])

Pipeline(steps=[('purchases_selector', FeatureSelector(column='purchases')),
                ('purchases_tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [37]:
y_predict2 = pipeline2.predict(X_test)
evaluate_results(y_test, y_predict2)

Classification results:
f1: 83.97%
recall: 100.00%
precision: 72.37%


In [39]:
pd.DataFrame({'metrics': ['f1', 'recall', 'precision'], 'Default learning': ['0.963', '0.932', '0.9954'], 'PU learning': ['0.8397', '1.00', '0.7237']})

,metrics,Default learning,PU learning
0,f1,0.963,0.8397
1,recall,0.932,1.00
2,precision,0.9954,0.7237
